# TP2-3

In [1]:
from collections import namedtuple
import os

In [16]:
def sha1(v):
    return hashlib.sha1(v).digest()

def sha224(v):
    return hashlib.sha3_224(v).digest()

def sha256(v):
    return hashlib.sha3_256(v).digest()

def sha384(v):
    return hashlib.sha3_384(v).digest()

def sha512(v):
    return hashlib.sha3_512(v).digest()

BFSecParam = namedtuple('SecParam', 'bq bp hashfn')

class BFSecParams:
    n1024 = BFSecParam(160,512,sha1)
    n2048 = BFSecParam(224,1024,sha224)
    n3072 = BFSecParam(256,1536,sha256)
    n7680 = BFSecParam(384,3840,sha384)
    n15360 = BFSecParam(512,7680,sha512)


class BF:

    def __init__(self, params):
        bq = params.bq
        bp = params.bp
        
        q = random_prime(2^bq-1,lbound=2^(bq-1))
        
        t = q*3*2^(bp - bq)
        while not is_prime(t-1):
            t = t << 1
        
        p = t - 1
        if not is_prime(p):
            raise Exception("not prime")
        
        # Aneis e Corpos
        Fp     = GF(p)                  # corpo primo com "p" elementos
        R.<z>  = Fp[]                   # anel dos polinomios em "z" de coeficientes em Fp
        f     = R(z^2 + z + 1)
        Fp2.<z> = GF(p^2, modulus=f)   
        
        # Curvas Elipticas supersingulares em Sagemath
        
        # a curva supersingular sobre Fp2  definida pela equação  y^2 = x^3 + 1
        E2 = EllipticCurve(Fp2, [0,1])
        
        # ponto arbitrário  de ordem "q" em E2        
        cofac = (p + 1)//q
        G = cofac * E2.random_point()
    
        s = 0
        while s<2 or s>q-1:
            rb = os.urandom(bq//8)
            s = int.from_bytes(rb, 'little')
        G_pub = s*G
        
        self.s = s
        self.public = G_pub
        self.G = G
        self.params = params
        self.E2 = E2
        self.Fp2 = Fp2
        self.Fp = Fp

    def key_extract(self, id):
        d = self.params.hashfn(id) * self.G
        return self.s*d
        

# extensão de Fp de dimensão 2 cujo módulo é o polinómio "f"
# o polinómio "f"  é irredutivel, tem grau 2 e verifica  z^3 = 1 mod f
# se o ponto (x,y) verificar a equação y^2 = x^3 + 1, 
#      então o ponto (z*x,y) verifica a mesma equação

def trace(x):       # função linear que mapeia Fp2  em  Fp
    return x + x^p

# emparelhamento e oraculo DDHP
def phi(P):             # a isogenia que mapeia  (x,y)  ->  (z*x,y)
    (x,y) = P.xy()
    return E2(z*x,y)

def TateX(P,Q,l=1):      # o emparelhamento de Tate generalizado
    return P.tate_pairing(phi(Q), q, 2)^l

def ddhp(P,Q,R):        # o oraculo DDHP  que decide se (P,Q,R) é um triplo de DH
    return tateX(P,Q) == tateX(R,G)

In [15]:
bf = BF(BFSecParams.n2048)

NameError: name 'public' is not defined